In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
Train_text = pd.read_csv("/content/drive/MyDrive/nlp-getting-started/train.csv")
Test_text = pd.read_csv("/content/drive/MyDrive/nlp-getting-started/test.csv")

In [4]:
Train_text

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
Train_text = Train_text[['text' , 'target']]
Train_text

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


Pre-processing

In [9]:
!pip install unidecode
!pip install word2number
!pip install contractions
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 8.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=b4a5297bc36e246d6a0083a2e8137d6bad84581153920d25fdae8bfd4aa29dd1
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 8.2 MB/s 
     |████████████████████████████████| 106 kB 54.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.5 MB 1.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.l

In [10]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions# load spacy model, can be "en_core_web_sm" as well
nlp = spacy.load('en_core_web_md')

In [11]:
# def remove_accented_chars(text):
#     """remove accented characters from text, e.g. café"""
#     text = unidecode.unidecode(text)
#     return text


# def expand_contractions(text):
#     """expand shortened words, e.g. don't to do not"""
#     text = contractions.fix(text)
#     return text

# def convert_numric_num_text(text):
#   doc = nlp(text)
#   tokens = [w2n.word_to_num(token.text) if token.pos_ == 'NUM' else token for token in doc]        


# def delete_Stop_words(text):
#   deselect_stop_words = ['no', 'not']
#   for w in deselect_stop_words:
#       nlp.vocab[w].is_stop = False  

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
deselect_stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
# exclude words from spacy stopwords list
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


def remove_html(text):
    Beau_text = BeautifulSoup(text, "html.parser")
    text_pro = Beau_text.get_text(separator=" ")
    return text_pro

def remove_whitespace(text):
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text


def expand_words(text):
    text = contractions.fix(text)
    return text

def convert_lower_Case(text):
    text = text.lower()

def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
  
    if remove_html == True: 
        text = remove_html(text)

    if extra_whitespace == True: 
        text = remove_whitespace(text)

    if accented_chars == True:
        text = remove_accented_chars(text)
    if contractions == True: 
        text = expand_words(text)

    if lowercase == True: 
        text = convert_lower_Case(text)

    # Here we should convert text to tokens
    doc = nlp(text) 

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text

        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) and flag == True:
            flag = False
        
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
         
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return clean_text

In [14]:
Train_text["text"].shape[0]

7613

In [15]:
for i in range(Train_text["text"].shape[0]):
   Train_text.iloc[i , 0] = ' '.join(text_preprocessing(Train_text.iloc[i , 0]))


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [16]:
Train_text["text"]

0       our deed be the reason of this earthquake alla...
1                 forest fire near la ronge sask . canada
2       all resident ask to shelter in place be be not...
3       people receive wildfire evacuation order in ca...
4       just got send this photo from ruby alaska as s...
                              ...                        
7608    giant crane hold a bridge collapse into nearby...
7609    @aria_ahrary @thetawni the out of control wild...
7610    m1.94 [ utc]?5 km s of volcano hawaii http://t...
7611    police investigate after an e - bike collide w...
7612    the late more home raze by northern california...
Name: text, Length: 7613, dtype: object

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=15000 ,oov_token='UNK')
tok.fit_on_texts(Train_text["text"])


In [18]:
print(tok.word_index['UNK'])
print(len(tok.word_index))
length_of_word_index = len(tok.word_index)

1
18985


In [19]:
from sklearn.model_selection import train_test_split
target=Train_text["target"]
train_words, validation_words ,train_target ,validation_target = train_test_split(Train_text['text'],target,test_size=0.2)

In [20]:
train_words_binary = tok.texts_to_matrix(train_words, mode='binary')
validation_words_binary = tok.texts_to_matrix(validation_words, mode='binary')

In [21]:
print(train_words.shape , train_target.shape)
print(validation_words.shape , validation_target.shape)


(6090,) (6090,)
(1523,) (1523,)


In [22]:
from keras import layers,models

model=models.Sequential()
model.add(layers.Dense(128,activation='relu',input_shape=(15000,)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [23]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras
model.compile(optimizer= RMSprop(lr=2e-5),
              loss= "binary_crossentropy",
              metrics= ["binary_accuracy"])

model.fit(train_words_binary,train_target,epochs=10,validation_data=(validation_words_binary,validation_target))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
191/191 [==============================] - 6s 28ms/step - loss: 0.6900 - binary_accuracy: 0.5667 - val_loss: 0.6861 - val_binary_accuracy: 0.6356
Epoch 2/10
191/191 [==============================] - 5s 25ms/step - loss: 0.6792 - binary_accuracy: 0.7110 - val_loss: 0.6752 - val_binary_accuracy: 0.6973
Epoch 3/10
191/191 [==============================] - 4s 23ms/step - loss: 0.6649 - binary_accuracy: 0.7435 - val_loss: 0.6613 - val_binary_accuracy: 0.7131
Epoch 4/10
191/191 [==============================] - 4s 22ms/step - loss: 0.6473 - binary_accuracy: 0.7585 - val_loss: 0.6450 - val_binary_accuracy: 0.7223
Epoch 5/10
191/191 [==============================] - 5s 25ms/step - loss: 0.6267 - binary_accuracy: 0.7662 - val_loss: 0.6267 - val_binary_accuracy: 0.7295
Epoch 6/10
191/191 [==============================] - 4s 22ms/step - loss: 0.6036 - binary_accuracy: 0.7749 - val_loss: 0.6070 - val_binary_accuracy: 0.7334
Epoch 7/10
191/191 [==============================] - 5s 2

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [25]:
train_words_sequnces = tok.texts_to_sequences(train_words)
validation_words_sequnces = tok.texts_to_sequences(validation_words)

In [26]:
train_words_sequnces = pad_sequences(train_words_sequnces, maxlen=25, padding='post',truncating='post')
validation_words_sequnces = pad_sequences(validation_words_sequnces, maxlen=25, padding='post', truncating='post')

In [28]:
model_1=models.Sequential()
model_1.add(layers.Embedding(length_of_word_index , 256))
model_1.add(layers.LSTM(128))
model_1.add(layers.Dense(64,activation='relu'))
model_1.add(layers.Dense(1,activation='sigmoid'))

model_1.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

model_1.fit(train_words_sequnces,train_target,epochs=10,validation_data=(validation_words_sequnces,validation_target))

Epoch 1/10
191/191 [==============================] - 21s 97ms/step - loss: 0.6880 - acc: 0.5639 - val_loss: 0.6822 - val_acc: 0.5877
Epoch 2/10
191/191 [==============================] - 15s 79ms/step - loss: 0.6816 - acc: 0.5660 - val_loss: 0.6748 - val_acc: 0.5877
Epoch 3/10
191/191 [==============================] - 16s 84ms/step - loss: 0.6743 - acc: 0.5655 - val_loss: 0.6649 - val_acc: 0.5916
Epoch 4/10
191/191 [==============================] - 15s 81ms/step - loss: 0.6571 - acc: 0.5869 - val_loss: 0.6400 - val_acc: 0.6244
Epoch 5/10
191/191 [==============================] - 17s 90ms/step - loss: 0.6139 - acc: 0.6826 - val_loss: 0.5994 - val_acc: 0.7052
Epoch 6/10
191/191 [==============================] - 22s 115ms/step - loss: 0.5688 - acc: 0.7376 - val_loss: 0.5753 - val_acc: 0.7216
Epoch 7/10
191/191 [==============================] - 19s 101ms/step - loss: 0.5264 - acc: 0.7553 - val_loss: 0.5455 - val_acc: 0.7400
Epoch 8/10
191/191 [==============================] - 22s 11

In [29]:
model_2=models.Sequential()
model_2.add(layers.Embedding(length_of_word_index , 256))
model_2.add(layers.GRU(128))
model_2.add(layers.Dense(64,activation='relu'))
model_2.add(layers.Dense(1,activation='sigmoid'))

model_2.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

model_2.fit(train_words_sequnces,train_target,epochs=10,validation_data=(validation_words_sequnces,validation_target))

Epoch 1/10
191/191 [==============================] - 19s 75ms/step - loss: 0.6888 - acc: 0.5553 - val_loss: 0.6834 - val_acc: 0.5844
Epoch 2/10
191/191 [==============================] - 18s 96ms/step - loss: 0.6859 - acc: 0.5654 - val_loss: 0.6815 - val_acc: 0.5877
Epoch 3/10
191/191 [==============================] - 19s 98ms/step - loss: 0.6852 - acc: 0.5662 - val_loss: 0.6804 - val_acc: 0.5877
Epoch 4/10
191/191 [==============================] - 18s 96ms/step - loss: 0.6848 - acc: 0.5662 - val_loss: 0.6796 - val_acc: 0.5883
Epoch 5/10
191/191 [==============================] - 15s 80ms/step - loss: 0.6844 - acc: 0.5667 - val_loss: 0.6792 - val_acc: 0.5890
Epoch 6/10
191/191 [==============================] - 19s 101ms/step - loss: 0.6835 - acc: 0.5668 - val_loss: 0.6779 - val_acc: 0.5883
Epoch 7/10
191/191 [==============================] - 19s 102ms/step - loss: 0.6824 - acc: 0.5668 - val_loss: 0.6766 - val_acc: 0.5909
Epoch 8/10
191/191 [==============================] - 19s 98

In [27]:
model_3 = models.Sequential()
model_3.add(layers.Embedding(length_of_word_index , 256))
model_3.add(layers.SpatialDropout1D(0.4))
model_3.add(layers.Bidirectional(layers.LSTM(196, dropout=0.05, recurrent_dropout=0.2 )))
model_3.add(layers.Dense(1, activation='sigmoid'))

model_3.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

model_3.fit(train_words_sequnces,train_target,epochs=10,validation_data=(validation_words_sequnces,validation_target))

Epoch 1/10
191/191 [==============================] - 79s 379ms/step - loss: 0.6841 - acc: 0.5655 - val_loss: 0.6747 - val_acc: 0.5877
Epoch 2/10
191/191 [==============================] - 71s 374ms/step - loss: 0.6741 - acc: 0.5665 - val_loss: 0.6659 - val_acc: 0.5896
Epoch 3/10
191/191 [==============================] - 83s 433ms/step - loss: 0.6643 - acc: 0.5764 - val_loss: 0.6564 - val_acc: 0.6100
Epoch 4/10
191/191 [==============================] - 70s 369ms/step - loss: 0.6513 - acc: 0.6212 - val_loss: 0.6417 - val_acc: 0.6382
Epoch 5/10
191/191 [==============================] - 69s 360ms/step - loss: 0.6268 - acc: 0.6696 - val_loss: 0.6107 - val_acc: 0.6986
Epoch 6/10
191/191 [==============================] - 75s 394ms/step - loss: 0.5770 - acc: 0.7143 - val_loss: 0.5790 - val_acc: 0.6999
Epoch 7/10
191/191 [==============================] - 80s 422ms/step - loss: 0.5436 - acc: 0.7302 - val_loss: 0.5578 - val_acc: 0.7236
Epoch 8/10
191/191 [==============================] - 6